In [2]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [5]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [10]:
year = 2021 
month = 2
taxi_type = 'fhv'

input_file = f"s3://nyc-tlc/trip data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
df = read_data(input_file)

In [11]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

## Q1. Notebook

### What's the mean predicted duration for this dataset?

In [15]:
y_pred.mean()

16.191691679979066

Answer: 16.19

## Q2. Preparing the output

In [18]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [22]:
output_file = f"output/fhv/{year:04d}_{month:02d}.parquet"
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

### What's the size of the output file?

In [26]:
! ls -la output/fhv/

total 19260
drwxrwxr-x 2 fdelca fdelca     4096 jun 28 15:23 .
drwxrwxr-x 3 fdelca fdelca     4096 jun 28 15:22 ..
-rw-rw-r-- 1 fdelca fdelca 19711507 jun 28 15:23 2021_02.parquet


Answer: 19M

## Q4. Virtual environment

### What's the first hash for the Scikit-Learn dependency? 

In [1]:
first_hash = "sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b"
first_hash

'sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b'

## Q5. Parametrize the script

### Run the script for March 2021. What's the mean predicted duration?

Mean result: 16.298821614015107

## Q6. Docker container

Mean result: 9.967573179784523